3. [Testing data sampling scheme](#Testing-data-sampling-scheme)

# Testing data sampling scheme

In [1]:
behav_data_f = 'Phenotypic_V1_0b_preprocessed1.csv'

In [2]:
df = pd.read_csv(behav_data_f)
df = df.loc[df['func_perc_fd'].notnull(), :]
df = df.loc[df['FILE_ID']!='no_filename', :]
df['AGE_YRS'] = np.floor(df['AGE_AT_SCAN'])
df.head()

NameError: name 'pd' is not defined

In [ ]:
motion_thresh = 10

In [ ]:
# Start by removing all participants whose data is below a certain
# motion threshold.
df_samp_motion = df.loc[df['func_perc_fd'] < motion_thresh, :]
df_samp_motion.head()

In [ ]:
plt.figure(1)
plt.subplot(121)
plt.hist(df_samp_motion.func_perc_fd, range=(0,100), bins=100)
plt.subplot(122)
blah = plt.hist(df.func_perc_fd, range=(0,100), bins=100)

In [ ]:
age_l=10
age_u=age_l+3
# Then remove participants who are younger (in years) than age_l and older
# than age_u. Note that this means people who are age_l and age_u
# (eg 6 and 10) will be included in the sample.
df_samp = df_samp_motion.loc[(df_samp_motion['AGE_YRS']>=age_l)
                                & (df_samp_motion['AGE_YRS']<=age_u), :]
plt.figure(1)
plt.subplot(121)
plt.hist(df_samp.AGE_YRS, range=(0,100), bins=100)
plt.subplot(122)
blah = plt.hist(df.AGE_YRS, range=(0,100), bins=100)

In [ ]:
plt.figure(1)
plt.subplot(121)
plt.scatter(df_samp.AGE_YRS, df_samp.func_perc_fd, alpha=.2)
plt.ylim(0, 100)
plt.xlim(0, 70)
plt.subplot(122)
plt.ylim(0, 100)
plt.xlim(0, 70)
blah = plt.scatter(df.AGE_YRS, df.func_perc_fd, alpha=.2)

In [ ]:
n=30
# Shuffle these remaining participants to ensure you get different sub
# samples each time you run the code.
df_samp_rand = df_samp.reindex(np.random.permutation(df_samp.index))

# Only keep the top 2*n participants.
df_samp_2n = df_samp_rand.iloc[:2*n, :]

# Sort these participants according to the sort columns of interest
sort_column_list = ['DSM_IV_TR', 'DX_GROUP', 'SITE_ID', 'SEX', 'AGE_YRS']
df_samp_2n_sorted = df_samp_2n.sort_values(by=sort_column_list)

# Now put all even numbered participants in group A and all odd numbered
# participants in group B.
df_grp_A = df_samp_2n_sorted.iloc[::2, :]
df_grp_B = df_samp_2n_sorted.iloc[1::2, :]

In [ ]:
(df_grp_A.subject.shape,
df_grp_B.subject.shape)

In [ ]:
n=150
# Shuffle these remaining participants to ensure you get different sub
# samples each time you run the code.
df_samp_rand = df_samp.reindex(np.random.permutation(df_samp.index))

# Only keep the top 2*n participants.
df_samp_2n = df_samp_rand.iloc[:2*n, :]

# Sort these participants according to the sort columns of interest
sort_column_list = ['DSM_IV_TR', 'DX_GROUP', 'SITE_ID', 'SEX', 'AGE_YRS']
df_samp_2n_sorted = df_samp_2n.sort_values(by=sort_column_list)

# Now put all even numbered participants in group A and all odd numbered
# participants in group B.
df_grp_A = df_samp_2n_sorted.iloc[::2, :]
df_grp_B = df_samp_2n_sorted.iloc[1::2, :]

In [ ]:
(df_grp_A.subject.shape[0],
df_grp_B.subject.shape[0])

It appears that, at least in this slice (age 10-13, < 10% bad volumes), we only have a maximum of 94\*2=188 subjects to work with, and so every iteration with these cutoffs, we're just remixing this subsample. As we increase our motion cuttoff, or rather reduce its stringency, we increase our available sampling pool

In [ ]:
def split_two_matched_samples(df, motion_thresh, age_l, age_u, n):
    """
    This function takes in a data frame, thresholds it to only include
    participants whose percentage bad frames are less than motion_thresh
    and participants who are between the lower and upper age limits (inclusive),
    then returns two matched samples of size n. The samples are matched on
    age in years, autism diagnosis, gender and scanning site.
    Information about the motion measure is here:
    http://preprocessed-connectomes-project.org/quality-assessment-protocol/
    """

    # Start by removing all participants whose data is below a certain
    # motion threshold.
    df_samp_motion = df.loc[df['func_perc_fd'] < motion_thresh, :]

    # Then remove participants who are younger (in years) than age_l and older
    # than age_u. Note that this means people who are age_l and age_u
    # (eg 6 and 10) will be included in the sample.
    df_samp = df_samp_motion.loc[(df_samp_motion['AGE_YRS']>=age_l)
                                    & (df_samp_motion['AGE_YRS']<=age_u), :]

    # Shuffle these remaining participants to ensure you get different sub
    # samples each time you run the code.
    df_samp_rand = df_samp.reindex(np.random.permutation(df_samp.index))

    # Only keep the top 2*n participants.
    df_samp_2n = df_samp_rand.iloc[:2*n, :]

    # Sort these participants according to the sort columns of interest
    sort_column_list = ['DSM_IV_TR', 'DX_GROUP', 'SITE_ID', 'SEX', 'AGE_YRS']
    df_samp_2n_sorted = df_samp_2n.sort_values(by=sort_column_list)

    # Now put all even numbered participants in group A and all odd numbered
    # participants in group B.
    df_grp_A = df_samp_2n_sorted.iloc[::2, :]
    df_grp_B = df_samp_2n_sorted.iloc[1::2, :]

    # Boom! Return these two data frames
    return df_grp_A, df_grp_B


Let's look at how often a particular subject ID re-occurs at different sample sizes.

In [ ]:
motion_thresh = 10; age_l = 10; age_u = age_l + 3
n_perms = 500

proportionDF = pd.DataFrame()

for motion_thresh in range(5, 55, 5):
    for n in range(30, 330, 30):
        subs = None
        for i in range(n_perms):
            #create two matched samples split on motion_thresh, age upper, age lower, and n
            df_A, df_B = split_two_matched_samples(df, motion_thresh, age_l, age_u, n)
            subs = pd.concat([subs, df_A.loc[:, 'subject'], df_B.loc[:, 'subject']])
        proportionDF = proportionDF.append(
            {'motion_thresh' : motion_thresh, 
             'N_set' : n*2,
             'N_obs' : subs.shape[0]/n_perms,
             'prop_min' : float(subs.value_counts().min())/n_perms,
             'prop_max' : float(subs.value_counts().max())/n_perms},
            ignore_index=True)

In [ ]:
propPlotFig, propPlotAx = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True)
propPlotFig.set_size_inches(10, 5)

labels = []
for N_set in range(30*2, 330*2, 30*2):
    tmpPropDF = proportionDF.query("N_set == " + str(N_set))
    propPlotAx[0].scatter(tmpPropDF.motion_thresh, tmpPropDF.prop_min, alpha=.2)
    propPlotAx[0].plot(tmpPropDF.motion_thresh, tmpPropDF.prop_min)
    propPlotAx[1].scatter(tmpPropDF.motion_thresh, tmpPropDF.prop_max, alpha=.2)
    propPlotAx[1].plot(tmpPropDF.motion_thresh, tmpPropDF.prop_max)
    labels.append(N_set)

handles, labels_trash = propPlotAx[1].get_legend_handles_labels()
propPlotAx[0].set_ylim([-.03, 1.03])
propPlotAx[0].set_xlabel('Motion Cutoff')
propPlotAx[0].xaxis.set_label_coords(1.1, -.075)
propPlotAx[0].set_ylabel('Proportion of Samples in which subject appears\nMinimum')
propPlotAx[1].set_ylabel('\nMaximum')
propPlotAx[1].legend(handles, labels, ncol=1, loc='center left',
                   title = "Set Sample Size", 
           bbox_to_anchor=[1, .5], 
           columnspacing=1.0, labelspacing=0.0,
           handletextpad=0.0, handlelength=1.5,
           fancybox=False, shadow=False)

In [ ]:
propPlotFig, propPlotAx = plt.subplots(nrows=1, ncols=1, sharex=True, sharey=True)
propPlotFig.set_size_inches(10, 5)

labels = []
for N_set in range(30*2, 330*2, 30*2):
    tmpPropDF = proportionDF.query("N_set == " + str(N_set))
    propPlotAx.scatter(tmpPropDF.motion_thresh, tmpPropDF.N_obs, alpha=.2)
    propPlotAx.plot(tmpPropDF.motion_thresh, tmpPropDF.N_obs)
    labels.append(N_set)

handles, labels_trash = propPlotAx.get_legend_handles_labels()
propPlotAx.set_xlabel('Motion Cutoff')
propPlotAx.set_ylabel('Observed Sample Size')
propPlotAx.legend(handles, labels, ncol=1, loc='center left',
                   title = "Set Sample Size", 
           bbox_to_anchor=[1, .5], 
           columnspacing=1.0, labelspacing=0.0,
           handletextpad=0.0, handlelength=1.5,
           fancybox=False, shadow=False)

## Conclusion

We're not actually able to iterate over sample size very well. Also, at low motion cutoffs, and high sample sizes, we're merely shuffling groups in each iteration. This might be helped by looking at a single, wide age range (the above is just ages 10-13), or bootstrap sampling with replacement.